
# AI-Assisted Literature Reviews  
*Session 5: Literature Review Automation *




## 🧪 Introduction

In this session, we build draft mini‑literature reviews with the OpenAI API.
Think of it as a head start: the model helps outline and write a first pass, which you will later verify and polish.

Typical workflow: Query → Retrieve → Synthesize → Cite
- Today we focus on Synthesize: producing structured, readable text quickly.
- Retrieval (Session 2 & 3) is important for accuracy. Here, we emphasize drafting and structure.
- Always validate claims with primary sources before using.

What you'll do
- Demo: Generate a mini‑review for “Machine learning in drug discovery”.
- Exercise: Try your own chemistry topic (e.g., catalysis, NMR, materials).
- Use prompt hints to adjust scope and style.



## ✅ Setup (Run this once)

Installs lightweight libraries and imports everything we need.


In [ ]:

%pip -q install openai ipywidgets reportlab

import os, textwrap, datetime   #add today’s date to reports.
import ipywidgets as widgets
from IPython.display import display, Markdown

from openai import OpenAI

from reportlab.lib.pagesizes import LETTER
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib.utils import simpleSplit



## 🔑 Add Your OpenAI API Key

Paste your key and click Save Key. If omitted, a placeholder draft is shown for demo purposes.


In [ ]:

api_key_box = widgets.Password(description='API Key:', placeholder='sk-...', layout=widgets.Layout(width='40%'))
save_btn = widgets.Button(description='Save Key', button_style='primary')
status = widgets.HTML("")

def save_key(_):
    if api_key_box.value.strip():
        os.environ["OPENAI_API_KEY"] = api_key_box.value.strip()
        status.value = "<span style='color:green'>Key saved for this session.</span>"
    else:
        status.value = "<span style='color:#B00020'>Please paste a valid key.</span>"

save_btn.on_click(save_key)
display(widgets.HBox([api_key_box, save_btn]), status)



## 🧩 Helpers (small & commented)

- generate_outline(topic, scope_hint, style_hint) — a bullet outline suitable for a mini‑review.
- generate_draft(topic, outline, scope_hint, style_hint) — expands outline into a short draft with headings.
- save_pdf(text, filename) — export the draft as a PDF.


In [ ]:

def _client():
    key = os.environ.get("OPENAI_API_KEY")
    return OpenAI(api_key=key) if key else None

SYSTEM_OUTLINE = (
    "You are a helpful assistant for chemists. Create a SHORT, logical outline for a mini literature review. "
    "Keep bullets concise and organized; avoid citations, focus on structure."
)

SYSTEM_DRAFT = (
    "You are a helpful assistant for chemists. Expand an outline into a SHORT, well-structured mini literature review. "
    "Use clear section headings, concise paragraphs, and neutral tone. Do not fabricate citations."
)

def _outline_prompt(topic: str, scope_hint: str = "", style_hint: str = "") -> str:
    today = datetime.date.today().isoformat()
    scope = f" Scope: {scope_hint.strip()}" if scope_hint else ""
    style = f" Style: {style_hint.strip()}" if style_hint else ""
    return f"""
Topic: {topic}
Date: {today}
Goal: Produce a SHORT outline for a mini literature review (6-10 bullets).{scope}{style}

Sections to include (as bullets):
- Brief background / motivation
- Key subareas or methods
- Representative applications or case studies
- Common datasets or evaluation metrics (if applicable)
- Challenges and limitations
- Emerging directions

Keep bullet points terse (max ~15 words each).
"""

def _draft_prompt(topic: str, outline_text: str, scope_hint: str = "", style_hint: str = "") -> str:
    today = datetime.date.today().isoformat()
    scope = f" Scope: {scope_hint.strip()}" if scope_hint else ""
    style = f" Style: {style_hint.strip()}" if style_hint else ""
    return f"""
Topic: {topic}
Date: {today}

Expand the following OUTLINE into a SHORT mini literature review (~300-500 words).{scope}{style}

OUTLINE:
{outline_text}

Rules:
- Use section headings (e.g., Background, Methods, Applications, Challenges, Outlook).
- Use concise, readable paragraphs for chemists with light coding experience.
- Avoid citations/DOIs; instead write placeholders like [cite] or [check source] where appropriate.
- Keep claims cautious; mark uncertain points with "verify".
"""

def generate_outline(topic: str, scope_hint: str = "", style_hint: str = "", model: str = "gpt-4o-mini") -> str:
    client = _client()
    prompt = _outline_prompt(topic, scope_hint, style_hint)
    if client is None:
        return textwrap.dedent(f"""
        Outline (demo mode) for topic: {topic}
        • Background and motivation [verify]
        • Major methods or subareas [verify]
        • Representative applications [verify]
        • Common datasets or metrics [verify]
        • Challenges and limitations [verify]
        • Emerging directions [verify]
        """)
    try:
        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": SYSTEM_OUTLINE},
                {"role": "user", "content": prompt},
            ],
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"*API error (outline):* {e}"

def generate_draft(topic: str, outline_text: str, scope_hint: str = "", style_hint: str = "", model: str = "gpt-4o-mini") -> str:
    client = _client()
    prompt = _draft_prompt(topic, outline_text, scope_hint, style_hint)
    if client is None:
        return textwrap.dedent(f"""
        Draft (demo mode) for topic: {topic}

        Background
        Short background paragraph... [verify]

        Methods
        Outline key approaches... [verify]

        Applications
        Examples and case studies... [verify]

        Challenges
        Limitations and pitfalls... [verify]

        Outlook
        Emerging directions... [verify]
        """)
    try:
        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": SYSTEM_DRAFT},
                {"role": "user", "content": prompt},
            ],
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"*API error (draft):* {e}"

def save_pdf(text: str, filename: str) -> str:
    path = f"{filename}"
    c = canvas.Canvas(path, pagesize=LETTER)
    width, height = LETTER

    x_margin = 0.8 * inch
    y_margin = 0.8 * inch
    max_width = width - 2*x_margin
    y = height - y_margin

    c.setFont("Times-Roman", 12)
    lines = []
    for paragraph in text.splitlines():
        wrapped = simpleSplit(paragraph, "Times-Roman", 12, max_width)
        lines.extend(wrapped if wrapped else [""])

    for line in lines:
        if y < y_margin:
            c.showPage()
            c.setFont("Times-Roman", 12)
            y = height - y_margin
        c.drawString(x_margin, y, line)
        y -= 14

    c.save()
    return path



## 💡 Demo — Mini Review in Two Steps

Topic: "Machine learning in drug discovery"

1) Click Make Outline to create a short bullet outline.
2) Click Write Draft to expand it into a mini‑review.
3) Optionally, Download PDF.


In [ ]:

topic_demo = widgets.Text(value="Machine learning in drug discovery", description="Topic:", layout=widgets.Layout(width="70%"))
scope_demo = widgets.Text(placeholder="Optional scope (e.g., small molecules, screening, 2018-2024)", description="Scope:")
style_demo = widgets.Text(placeholder="Optional style (e.g., clinical tone, more bullets)", description="Style:")

model_pick = widgets.Dropdown(
    options=[("gpt-4o-mini (fast)","gpt-4o-mini"),
             ("gpt-4o (quality)","gpt-4o"),
             ("gpt-3.5-turbo (legacy)","gpt-3.5-turbo")],
    value="gpt-4o-mini", description="Model:"
)

btn_outline = widgets.Button(description="Make Outline", button_style="info")
btn_draft   = widgets.Button(description="Write Draft", button_style="success")
btn_pdf     = widgets.Button(description="Download PDF", button_style="primary")

out_outline = widgets.Output()
out_draft   = widgets.Output()
out_pdf     = widgets.Output()

display(topic_demo, scope_demo, style_demo, model_pick,
        widgets.HBox([btn_outline, btn_draft, btn_pdf]),
        out_outline, out_draft, out_pdf)

_outline_cache = {"text": ""}

def on_make_outline(_):
    out_outline.clear_output(); out_draft.clear_output(); out_pdf.clear_output()
    with out_outline: display(Markdown("Generating outline..."))
    text = generate_outline(topic_demo.value, scope_demo.value, style_demo.value, model=model_pick.value)
    _outline_cache["text"] = text
    out_outline.clear_output()
    with out_outline: display(Markdown("### Outline")); display(Markdown(f"```{text}```"))

def on_write_draft(_):
    out_draft.clear_output(); out_pdf.clear_output()
    if not _outline_cache["text"].strip():
        with out_draft: display(Markdown("Please create an outline first.")); return
    with out_draft: display(Markdown("Expanding outline into a mini review..."))
    text = generate_draft(topic_demo.value, _outline_cache["text"], scope_demo.value, style_demo.value, model=model_pick.value)
    _outline_cache["draft"] = text
    out_draft.clear_output()
    with out_draft: display(Markdown("### Draft")); display(Markdown(f"```{text}```"))

def on_download_pdf(_):
    out_pdf.clear_output()
    draft = _outline_cache.get("draft","").strip()
    if not draft:
        with out_pdf: display(Markdown("No draft available. Please write the draft first.")); return
    safe = topic_demo.value.replace(" ","_")
    path = save_pdf(draft, f"MiniReview_{safe}.pdf")
    with out_pdf: display(Markdown(f"[Download PDF](sandbox:{path})"))

btn_outline.on_click(on_make_outline)
btn_draft.on_click(on_write_draft)
btn_pdf.on_click(on_download_pdf)



## ✍️ Exercise — Your Own Topic

Try a topic you care about (examples below). Use Scope to focus (e.g., "2019–2024, small molecules") and Style to change tone.

Example prompts
- Recent advances in catalysis for C–N coupling
- Applications of NMR analysis in fragment-based drug discovery
- Electrolyte design in solid-state batteries


In [ ]:

topic = widgets.Text(placeholder="e.g., Recent advances in catalysis for C–N coupling", description="Topic:", layout=widgets.Layout(width="70%"))
scope = widgets.Text(placeholder="Optional scope (e.g., years, subdomain)", description="Scope:")
style = widgets.Text(placeholder="Optional style (e.g., more bullets, lay tone)", description="Style:")

btn_outline2 = widgets.Button(description="Make Outline", button_style="info")
btn_draft2   = widgets.Button(description="Write Draft", button_style="success")
btn_pdf2     = widgets.Button(description="Download PDF", button_style="primary")

out_outline2 = widgets.Output()
out_draft2   = widgets.Output()
out_pdf2     = widgets.Output()

display(topic, scope, style, model_pick, widgets.HBox([btn_outline2, btn_draft2, btn_pdf2]), out_outline2, out_draft2, out_pdf2)

_outline_cache2 = {"text": ""}

def on_make_outline2(_):
    out_outline2.clear_output(); out_draft2.clear_output(); out_pdf2.clear_output()
    if not topic.value.strip():
        with out_outline2: display(Markdown("Please enter a topic.")); return
    with out_outline2: display(Markdown("Generating outline..."))
    text = generate_outline(topic.value, scope.value, style.value, model=model_pick.value)
    _outline_cache2["text"] = text
    out_outline2.clear_output()
    with out_outline2: display(Markdown("### Outline")); display(Markdown(f"```{text}```"))

def on_write_draft2(_):
    out_draft2.clear_output(); out_pdf2.clear_output()
    if not _outline_cache2["text"].strip():
        with out_draft2: display(Markdown("Please create an outline first.")); return
    with out_draft2: display(Markdown("Expanding outline into a mini review..."))
    text = generate_draft(topic.value, _outline_cache2["text"], scope.value, style.value, model=model_pick.value)
    _outline_cache2["draft"] = text
    out_draft2.clear_output()
    with out_draft2: display(Markdown("### Draft")); display(Markdown(f"```{text}```"))

def on_download_pdf2(_):
    out_pdf2.clear_output()
    draft = _outline_cache2.get("draft","").strip()
    if not draft:
        with out_pdf2: display(Markdown("No draft available. Please write the draft first.")); return
    safe = topic.value.replace(" ","_")
    path = save_pdf(draft, f"MiniReview_{safe}.pdf")
    with out_pdf2: display(Markdown(f"[Download PDF](sandbox:{path})"))

btn_outline2.on_click(on_make_outline2)
btn_draft2.on_click(on_write_draft2)
btn_pdf2.on_click(on_download_pdf2)



## 📘 Reflection
- Which parts of the draft would you verify against primary literature?
- What would a final review require (e.g., figures, DOIs, peer edits)?
